<a href="https://colab.research.google.com/github/BelovedTemitope/Machine-Learning-Projects/blob/master/DSN_2020_hackathon_Qualifying_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost

     |████████████████████████████████| 66.1MB 57kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
#import missingno as msno

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, roc_auc_score, auc, classification_report
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
#from sklearn.model_selection import KFold
#from paramsearch import paramsearch
#from itertools import product,chain
from sklearn.model_selection import GridSearchCV

In [ ]:
%matplotlib inline
pd.set_option('display.max_columns', 100)
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 10

In [ ]:
import random
seed = 4092
random.seed(seed)
np.random.seed(seed)

In [ ]:
train  = pd.read_csv('/content/drive/My Drive/DSN Hack data/Train.csv')
test = pd.read_csv('/content/drive/My Drive/DSN Hack data/Test.csv')

In [ ]:
train.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)

In [ ]:
test_id = test.Applicant_ID

In [ ]:
#Label Encoding target
train.default_status.replace({'no':0, 'yes':1}, inplace=True)

In [ ]:
features = train.columns.drop(['Applicant_ID', 'default_status'])
X = pd.get_dummies(train[features])
y = train.default_status
test = pd.get_dummies(test[features])

In [ ]:
# Import Scaler
stdscaler = StandardScaler()
minmaxscaler = MinMaxScaler()


In [ ]:
X_scaled = minmaxscaler.fit_transform(X)

In [ ]:
skfold = StratifiedKFold(n_splits=30, shuffle=True, random_state=419)

In [ ]:
# Models
cat_clf = CatBoostClassifier(random_state=419)
random_clf = RandomForestClassifier(random_state=419)
lgb_clf = LGBMClassifier(random_state=419)


In [ ]:
def stratified_vote():
  scores, preds = [], []
  i = 1
  for train_split, val_split in skfold.split(X_scaled, y):
    X_train, X_test, y_train, y_test = X.iloc[train_split], X.iloc[val_split], y.iloc[train_split], y.iloc[val_split]

    # Voting Classifier
    vote = VotingClassifier(estimators=[('cat_clf', cat_clf), ('random_clf', random_clf), ('lgb_clf', lgb_clf)], voting='soft')
    vote.fit(X_train, y_train)

    # Score
    score = roc_auc_score(y_test, vote.predict_proba(X_test)[:,-1])
    pred = vote.predict_proba(test)[:,-1]
    scores.append(score)
    preds.append(pred)
    print('ROC for {} split: '.format(i), score)
    i += 1
  
  print('Finam absolute ROC: ', np.mean(scores))
  final_predictions = np.mean(preds, axis=0)
  return final_predictions

In [ ]:
final_predictions = stratified_vote()

Streaming output truncated to the last 5000 lines.
147:	learn: 0.3979447	total: 6.05s	remaining: 34.8s
148:	learn: 0.3978798	total: 6.09s	remaining: 34.8s
149:	learn: 0.3977896	total: 6.13s	remaining: 34.7s
150:	learn: 0.3977019	total: 6.18s	remaining: 34.8s
151:	learn: 0.3976378	total: 6.22s	remaining: 34.7s
152:	learn: 0.3974752	total: 6.26s	remaining: 34.7s
153:	learn: 0.3973484	total: 6.3s	remaining: 34.6s
154:	learn: 0.3972112	total: 6.34s	remaining: 34.6s
155:	learn: 0.3971165	total: 6.39s	remaining: 34.6s
156:	learn: 0.3970098	total: 6.43s	remaining: 34.5s
157:	learn: 0.3969358	total: 6.47s	remaining: 34.5s
158:	learn: 0.3968204	total: 6.51s	remaining: 34.4s
159:	learn: 0.3967519	total: 6.55s	remaining: 34.4s
160:	learn: 0.3966837	total: 6.59s	remaining: 34.3s
161:	learn: 0.3965903	total: 6.63s	remaining: 34.3s
162:	learn: 0.3964712	total: 6.67s	remaining: 34.2s
163:	learn: 0.3963666	total: 6.71s	remaining: 34.2s
164:	learn: 0.3962503	total: 6.75s	remaining: 34.1s
165:	learn: 0.

In [ ]:
submission = pd.DataFrame({'Applicant_ID' : test_id, 'default_status' : final_predictions})

In [ ]:
submission.to_csv('vote.csv', index=False)